In [109]:
from bs4 import BeautifulSoup
import os
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [110]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [/Users/elliottlevy/.wdm/drivers/chromedriver/mac64/87.0.4280.88/chromedriver] found in cache


In [111]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)
html = browser.html
soup1 = BeautifulSoup(html, 'html.parser')

In [112]:
title = soup1.find_all('div', class_='content_title')[1].text
p = soup1.find_all('div', class_='article_teaser_body')[0].text


In [113]:
print(title)
print("----------------------------------")
print(p)

Celebrate the Perseverance Rover Landing With NASA's Student Challenge
----------------------------------
The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.


In [180]:
jpl_site = 'https://www.jpl.nasa.gov/'
image_site = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(image_site)
html = browser.html
soup2 = BeautifulSoup(html, 'html.parser')

In [181]:
a = soup2.find_all('a', class_='button fancybox')[0]["data-fancybox-href"]
a
my_image_url = jpl_site + a
print(my_image_url)


https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA18182_ip.jpg


In [182]:
tables = pd.read_html('https://space-facts.com/mars/')

In [183]:
mars_facts = tables[0]
mars_facts.columns = ['Fact','Answer']
mars_html_table = mars_facts.to_html()
mars_html_table = mars_html_table.replace('\n', '')

In [185]:
main_usgs = 'https://astrogeology.usgs.gov'
mars_usgs = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(mars_usgs)

hemispheres_html = browser.html

soup3 = BeautifulSoup(hemispheres_html, 'html.parser')

In [190]:
mars_hemispheres = soup3.find_all('div', class_='item')

hemisphere_image_urls = []

for i in mars_hemispheres:
    hemisphere = i.find('div', class_="description")
    title2 = hemisphere.h3.text
    
    hemisphere_link = hemisphere.a["href"]    
    browser.visit(main_usgs + hemisphere_link)
    
    image_html = browser.html
    image_soup = BeautifulSoup(image_html, 'html.parser')
    
    image_link = image_soup.find('div', class_='downloads')
    image_url = image_link.find('li').a['href']

    image_dict = {}
    image_dict['title'] = title2
    image_dict['img_url'] = image_url
    
    hemisphere_image_urls.append(image_dict)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]


In [193]:
mars_dict = {
        "news_title": title,
        "news_p": p,
        "featured_image_url": my_image_url,
        "fact_table": str(mars_html_table),
        "hemisphere_images": hemisphere_image_urls
    }
mars_dict

{'news_title': 'Valles Marineris Hemisphere Enhanced',
 'news_p': 'The rover touches down on the Red Planet next month, and students are invited to join the excitement by designing, building, and landing their own Mars mission. NASA can help.',
 'featured_image_url': 'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA18182_ip.jpg',
 'fact_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Fact</th>      <th>Answer</th>    </tr>  </thead>  <tbody>    <tr>      <th>0</th>      <td>Equatorial Diameter:</td>      <td>6,792 km</td>    </tr>    <tr>      <th>1</th>      <td>Polar Diameter:</td>      <td>6,752 km</td>    </tr>    <tr>      <th>2</th>      <td>Mass:</td>      <td>6.39 × 10^23 kg (0.11 Earths)</td>    </tr>    <tr>      <th>3</th>      <td>Moons:</td>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>4</th>      <td>Orbit Distance:</td>      <td>227,943,824 km (1.38 AU)</td>    </tr>    <tr>  

In [195]:
browser.quit()